<a href="https://colab.research.google.com/github/ramaoutlook/TF-IDF-Ecommorce-Item-Category-Classification/blob/main/Ecommerce_item_category_classification_using_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle==1.5.12

# Make a directory for kaggle and copy the kaggle.json to it
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/

# Set permissions for the json file
!chmod 600 ~/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73026 sha256=038a805dff2664b2369422adfcd2dbfcfc5227c88a7c6a6410a538d99325e0b8
  Stored in directory: /root/.cache/pip/wheels/2e/27/39/f44e52756a6407b444143f233abe9fda0e18a23e8b20e0cd1c
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.5
    Uninstalling kaggle-1.7.4.5:
      Successfully uninstalled kaggle-1.7.4.5


In [3]:
import pandas as pd
import kaggle

# Download the dataset
# Replace 'saurabhshahane/ecommerce-text-classification' with the dataset path from the URL
!kaggle datasets download -d saurabhshahane/ecommerce-text-classification

# Unzip the dataset
!unzip ecommerce-text-classification.zip



100% 7.86M/7.86M [00:00<00:00, 81.1MB/s]
100% 7.86M/7.86M [00:00<00:00, 80.7MB/s]
Archive:  ecommerce-text-classification.zip
  inflating: ecommerceDataset.csv    


In [4]:
# Load the dataset into a pandas DataFrame
# Replace 'ecommerce_text_classification.csv' with the actual name of the CSV file in the dataset
df = pd.read_csv('/content/ecommerceDataset.csv')

# Display the first few rows of the DataFrame
print(df.head())

   Household  \
0  Household   
1  Household   
2  Household   
3  Household   
4  Household   

  Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your 

In [5]:
df.head()

,Household,"Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some for eternal bliss.so bring home this elegant print that is lushed with rich colors that makes it nothing but sheer elegance to be to your friends and family.it would be treasured forever by whoever your lucky recipient is. Liven up your place with these intriguing paintings that are high definition hd graphic digital prints for home, office or any room."
0,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
1,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
2,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
3,Household,Incredible Gifts India Wooden Happy Birthday U...
4,Household,Pitaara Box Romantic Venice Canvas Painting 6m...


In [6]:
# prompt: df doesn't has the heading and it took first row data as headings. can you give some headings to df?

import pandas as pd
df = pd.read_csv('/content/ecommerceDataset.csv', header=None)
df.columns = ['label', 'text']

df.head()

,label,text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50425 entries, 0 to 50424
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   50425 non-null  object
 1   text    50424 non-null  object
dtypes: object(2)
memory usage: 788.0+ KB


In [8]:
df.label.value_counts()

,count
label,
Household,19313
Books,11820
Electronics,10621
Clothing & Accessories,8671


In [9]:
# Below code is to Balancing the labels to match the majority label's count with all the labels by increasing the
   # minority labels
from sklearn.utils import resample


# Get the max class size (Household)
max_size = df['label'].value_counts().max()

# Split data by class
df_books = df[df.label == 'Books']
df_electronics = df[df.label == 'Electronics']
df_clothing = df[df.label == 'Clothing & Accessories']
df_household = df[df.label == 'Household']  # Already has max count

# Upsample all except household
df_books_upsampled = resample(df_books, replace=True, n_samples=max_size, random_state=123)
df_electronics_upsampled = resample(df_electronics, replace=True, n_samples=max_size, random_state=123)
df_clothing_upsampled = resample(df_clothing, replace=True, n_samples=max_size, random_state=123)

# Combine all into one balanced dataframe
df_balanced = pd.concat([df_books_upsampled, df_electronics_upsampled, df_clothing_upsampled, df_household])

# Shuffle the final balanced dataset (optional but recommended)
df_balanced = df_balanced.sample(frac=1, random_state=123).reset_index(drop=True)

# Check result
print(df_balanced['label'].value_counts())


label
Household                 19313
Electronics               19313
Books                     19313
Clothing & Accessories    19313
Name: count, dtype: int64


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50425 entries, 0 to 50424
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   50425 non-null  object
 1   text    50424 non-null  object
dtypes: object(2)
memory usage: 788.0+ KB


In [11]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77252 entries, 0 to 77251
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   77252 non-null  object
 1   text    77252 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [12]:
df_balanced['label_num'] = df_balanced.label.map({'Books':1, 'Clothing & Accessories':3, 'Electronics':2, 'Household':0})
df_balanced.head()

,label,text,label_num
0,Household,Homesake Edison Filament Wooden Walnut Bubble ...,0
1,Household,Kitchen Bazaar Deluxe Fruit & Vegetable Manual...,0
2,Electronics,Logitech Wireless presenter R400 (Black) Produ...,2
3,Books,Differential Calculus For Beginners About the ...,1
4,Electronics,LG GH24NSB0 DVD Writer 24X SATA Internal OEM P...,2


In [13]:
df_balanced.tail(2)

,label,text,label_num
77250,Electronics,DMG Screen Protector for Apple iPad - Shockpro...,2
77251,Books,"The Great Escape – Health, Wealth, and the Ori...",1


In [14]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, = train_test_split(
      df_balanced['text'],
      df_balanced['label_num'],
      test_size=0.2,
      random_state=2022,
      stratify=df_balanced.label_num
  )

In [15]:
X_train.shape, X_test.shape

((61801,), (15451,))

In [16]:
y_train.value_counts()

,count
label_num,
2,15451
0,15450
1,15450
3,15450


In [17]:
y_test.value_counts()

,count
label_num,
1,3863
0,3863
3,3863
2,3862


In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

clf = Pipeline([
    ('vectorizer_tfidf',TfidfVectorizer()),
    ('KNN', KNeighborsClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      3863
           1       0.97      0.98      0.98      3863
           2       0.97      0.97      0.97      3862
           3       0.99      0.99      0.99      3863

    accuracy                           0.97     15451
   macro avg       0.97      0.97      0.97     15451
weighted avg       0.97      0.97      0.97     15451



In [19]:
X_test[:5]

,text
43453,AHUJA AWM-490V2 Dual PA Wireless Microphone(Bl...
25068,The Religions Book (Big Ideas) Review Fascinat...
29449,TAMILNADU HISTORY BOOKS (CLASS XI & XII) SET O...
34541,SIMPEX 522 Manual Shoe Mount Flash for DSLR Ca...
40766,Nova Plus Amaze NI-10 1100-Watt Dry Iron (Grey...


In [20]:
X_test[:5][43453]

"AHUJA AWM-490V2 Dual PA Wireless Microphone(Black) AHUJA introduces AWM-490V2 as an affordable high performance VHF wireless microphone system. It consists of two wireless microphones and a highly sensitive dual channel non-diversity receiver. This ensures an effective long distance coverage of more than 150 feet under ideal working conditions. Receiver operates on 9V DC through an AC Adaptor supplied along with the system. Each microphone operates with a 9V Dry Battery. Microphone with 3-position slide switch for 'Audio On'-'Audio Mute'- 'Off' positions. Audio output from receiver available at OUT 1 and OUT 2 individually and as mixed output at OUT 1 + 2. Ideal for: PA Applications, stage and auditoriums, seminar halls, places of worship and recreation centres. Microphone RF Output Power 15mW (max.) modulation mode FM (F3E) microphone element dynamic cardioid dynamic range 100dB spurious emission < 45dB maximum deviation A± 25kHz frequency response 50-15 000Hz antenna internal dipole

In [21]:
y_test[:5]

,label_num
43453,2
25068,1
29449,1
34541,2
40766,0


In [22]:
y_pred[:5]

array([2, 1, 1, 2, 0])

In [23]:
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline([
    ('vectorizer_tfidf',TfidfVectorizer()),
    ('Random Forest', RandomForestClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3863
           1       0.99      1.00      0.99      3863
           2       0.99      0.99      0.99      3862
           3       1.00      1.00      1.00      3863

    accuracy                           0.99     15451
   macro avg       0.99      0.99      0.99     15451
weighted avg       0.99      0.99      0.99     15451



In [24]:
from sklearn.naive_bayes import MultinomialNB

clf = Pipeline([
    ('vectorizer_tfidf',TfidfVectorizer()),
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      3863
           1       0.97      0.94      0.96      3863
           2       0.96      0.95      0.96      3862
           3       0.97      0.99      0.98      3863

    accuracy                           0.96     15451
   macro avg       0.96      0.96      0.96     15451
weighted avg       0.96      0.96      0.96     15451



In [25]:
## Utility function for pre-processing the text
import spacy

#load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
  #remove stop words and Lemmatize the text
  doc = nlp(text)
  filtered_tokens = []
  for token in doc:
    if token.is_stop or token.is_punct:
      continue
    filtered_tokens.append(token.lemma_)
  return " ".join(filtered_tokens)


In [26]:
df_balanced['preprocessed_txt'] = df_balanced['text'].apply(preprocess)

In [27]:
df_balanced.head()

,label,text,label_num,preprocessed_txt
0,Household,Homesake Edison Filament Wooden Walnut Bubble ...,0,Homesake Edison Filament Wooden Walnut bubble ...
1,Household,Kitchen Bazaar Deluxe Fruit & Vegetable Manual...,0,Kitchen Bazaar Deluxe Fruit Vegetable Manual J...
2,Electronics,Logitech Wireless presenter R400 (Black) Produ...,2,Logitech Wireless presenter r400 black product...
3,Books,Differential Calculus For Beginners About the ...,1,Differential Calculus Beginners Author editori...
4,Electronics,LG GH24NSB0 DVD Writer 24X SATA Internal OEM P...,2,LG GH24NSB0 DVD Writer 24X SATA Internal OEM P...


In [30]:
X_train, X_test, y_train, y_test, = train_test_split(
      df_balanced['preprocessed_txt'],
      df_balanced['label_num'],
      test_size=0.2,
      random_state=2022,
      stratify=df_balanced.label_num
)

In [31]:
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline([
    ('vectorizer_tfidf',TfidfVectorizer()),
    ('Random Forest', RandomForestClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3863
           1       0.99      1.00      0.99      3863
           2       0.99      0.99      0.99      3862
           3       1.00      1.00      1.00      3863

    accuracy                           0.99     15451
   macro avg       0.99      0.99      0.99     15451
weighted avg       0.99      0.99      0.99     15451



https://www.youtube.com/watch?v=ATK6fm3cYfI&list=PLeo1K3hjS3uuvuAXhYjV2lMEShq2UYSwX&index=18&pp=iAQB

codebasics 18th video of NLP Playlist